In [20]:
import pandas as pd
ticks = pd.read_csv ('ticks.csv', sep="|")
ticks['gvkey'] = ticks['gvkey'].fillna(0.0).astype(int)
ticks['Permco'] = ticks['Permco'].fillna(0.0).astype(int)
ticks

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647
...,...,...,...,...,...,...,...
107,"WERTHEIM SCHRODER & CO., INC.",Schroder Wertheim & Co. Inc.,NaN,6/24/1988,11/8/1990,0,0
108,WESTPAC POLLOCK GOV'T SECURITIES INC,Westpac Banking Corporation,WBK (NYSE),2/4/1987,6/27/1990,22027,15362
109,"WHITE, WELD & CO INC.","Merrill Lynch & Co., Inc.",NaN,2/26/1976,4/18/1978,21190,7267
110,"YAMAICHI INT'L (AMERICA), INC.",Yamaichi Securities Company Limited,NaN,9/29/1988,12/4/1997,0,0


In [3]:
test = pd.read_csv('test.csv', sep='|')
test['gvkey'] = test['gvkey'].fillna(0.0).astype(int)
test['Permco'] = test['Permco'].fillna(0.0).astype(int)
test

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243
2,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647
3,Banc One,JPMorgan Chase & Co. (Acquired by JPMorgan Chase),JPM (NYSE),4/1/1999,8/1/2004,20436,2968
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647
5,Bank of America,Bank of America Corporation,BAC (NYSE),5/17/1999,11/1/2010,3151,7647
6,Bank of Nova Scotia,The Bank of Nova Scotia (Scotiabank),"BNS (NYSE, TSX)",10/4/2011,Current,43264,15582
7,Bankers Trust,Deutsche Bank AG (Acquired by Deutsche Bank),DB (XETRA),5/19/1960,10/22/1997,0,0
8,Barclays,Barclays PLC,BARC (LSE),4/1/1998,Current,20269,12673
9,Barclays De Zoete Wedd,Barclays PLC,BARC (LSE),12/7/1989,6/30/1996,20269,12673


In [12]:
import pandas as pd
import wrds
import config
from datetime import datetime

db = wrds.Connection(wrds_username=config.WRDS_USERNAME)

Loading library list...
Done


In [33]:
def fetch_financial_data(pgvkey, start_date, end_date):
    """
    Fetch financial data for a given ticker and date range from the CCM database in WRDS.
    
    :param gvkey: The gvkey symbol for the company.
    :param start_date: The start date for the data in YYYY-MM-DD format.
    :param end_date: The end date for the data in YYYY-MM-DD format or 'Current'.
    :return: A DataFrame containing the financial data.
    """
    if not gvkey:  # Skip if no ticker is available
        return pd.DataFrame()
    
    # If the end date is 'Current', replace it with today's date
    if end_date == 'Current':
        end_date = datetime.today().strftime('%Y-%m-%d')
    
    # Convert dates to the correct format if necessary
    start_date = pd.to_datetime(start_date).strftime('%Y-%m-%d')
    end_date = pd.to_datetime(end_date).strftime('%Y-%m-%d')
    
        
    query = f"""
    SELECT datadate, at AS total_assets, lt AS book_debt, ceq AS book_equity, csho*prcc_f AS market_equity, gvkey, conm
    FROM comp.funda as cst
    WHERE cst.gvkey = '{str(pgvkey).zfill(6)}'
    AND cst.datadate BETWEEN '{start_date}' AND '{end_date}'
    AND indfmt='INDL'
    AND datafmt='STD'
    AND popsrc='D'
    AND consol='C'
    """
    data = db.raw_sql(query)
    return data

Get all data for primary dealers

In [42]:
empty_tickers = []
prim_dealers = pd.DataFrame()
# Iterate over DataFrame rows and fetch data for each ticker
for index, row in ticks.iterrows():
    gvkey = row['gvkey']
    start_date = row['Start Date']
    end_date = row['End Date']      # Formatting date for the query
    
    # Fetch financial data for the ticker if available
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['Ticker']:gvkey})
    else:
        prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
print(empty_tickers)

C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  prim_dealers = pd.concat([new_data, prim_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\3471547033.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is

[]


Get all data from banks

Get data from all firms in Compustat

In [43]:
prim_dealers.drop_duplicates()

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1993-12-31,50782.297,47213.898,3226.500,7121.765625,008007,WELLS FARGO & CO
1,1994-12-31,59315.898,55469.500,3334.400,7226.264375,008007,WELLS FARGO & CO
2,1995-12-31,72134.375,66822.250,5009.797,11641.080000,008007,WELLS FARGO & CO
3,1996-12-31,80175.375,74111.188,5875.398,16038.580500,008007,WELLS FARGO & CO
4,1997-12-31,88540.188,81518.000,6834.199,29396.486250,008007,WELLS FARGO & CO
...,...,...,...,...,...,...,...
3,2001-12-31,531713.000,517166.000,9767.000,24997.940000,015504,ABN-AMRO HOLDINGS NV
4,2002-12-31,582984.873,567686.209,10450.400,25655.008000,015504,ABN-AMRO HOLDINGS NV
5,2003-12-31,705982.489,684869.917,15411.170,38457.492840,015504,ABN-AMRO HOLDINGS NV
6,2004-12-31,823953.817,797891.813,19230.729,44400.773200,015504,ABN-AMRO HOLDINGS NV


In [5]:
def pull_CRSP_Comp_Link_Table(wrds_username=config.WRDS_USERNAME):
    sql_query = """
        SELECT 
            gvkey, lpermco AS permco, linktype, linkprim, linkdt, linkenddt, tic
        FROM 
            crsp.ccmxpf_linkhist
        WHERE 
            substr(linktype,1,1)='L' AND 
            (linkprim ='C' OR linkprim='P')
        """
    db = wrds.Connection(wrds_username=wrds_username)
    ccm = db.raw_sql(sql_query, date_cols=["linkdt", "linkenddt"])
    db.close()
    return ccm

In [ ]:
pull_CRSP_Comp_Link_Table()

In [35]:
linktable = pd.read_csv('linktable.csv')
linktable

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,1000,A & E PLASTIK PAK INC,AE.2,3089.0,P,01,LU,25881,23369,11/13/1970,6/30/1978
1,1001,A & M FOOD SERVICES INC,AMFD.,5812.0,P,01,LU,10015,6398,9/20/1983,7/31/1986
2,1002,AAI CORP,AAIC.1,3825.0,C,01,LC,10023,22159,12/14/1972,6/5/1973
3,1003,A.A. IMPORTING CO INC,ANTQ,5712.0,C,01,LU,10031,6672,12/7/1983,8/16/1989
4,1004,AAR CORP,AIR,5080.0,P,01,LU,54594,20000,4/24/1972,E
...,...,...,...,...,...,...,...,...,...,...,...
31944,349994,CLEARMIND MEDICINE INC,CMND,2834.0,P,01,LC,23514,59438,11/15/2022,E
31945,350681,GETNET ADQUIRENCIA E,GET,7374.0,P,90,LC,22205,58855,10/22/2021,3/31/2023
31946,351038,QUOIN PHARMACEUTICALS LTD,QNRX,2834.0,P,90,LC,16161,55612,10/29/2021,E
31947,352262,COOL COMPANY LTD,CLCO,4400.0,P,01,LC,23773,59507,3/17/2023,E


In [36]:
merged = pd.merge(ticks, linktable, left_on='gvkey', right_on='gvkey')
merged

,Primary Dealer,Holding Company,Ticker,Start Date,End Date,Permco,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
0,ABN AMRO INCORPORATED,NaN,ABN,9/29/1998,9/15/2006,31989,15504,ABN-AMRO HOLDINGS NV,ABNYY,6020.0,P,90,LC,84766,31989,5/21/1997,4/24/2008
1,"AUBREY G. LANSTON & CO., INC.",Citigroup Inc.,C (NYSE),5/19/1960,4/17/2000,20483,3243,CITIGROUP INC,C,6199.0,P,01,LC,70519,20483,10/29/1986,E
2,Citigroup,Citigroup Inc.,C (NYSE),6/15/1961,Current,20483,3243,CITIGROUP INC,C,6199.0,P,01,LC,70519,20483,10/29/1986,E
3,BA Securities,Bank of America Corporation,BAC (NYSE),4/18/1994,9/30/1997,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
4,Bank of America,Bank of America Corporation,BAC (NYSE),11/17/1971,4/15/1994,437,7647,BANK OF AMERICA CORP,BAC,6020.0,P,01,LC,59408,3151,12/14/1972,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"SMITH BARNEY, HARRIS UPHAM & CO.,INC",Primerica Corporation,PRI,8/22/1979,8/31/1998,22177,1414,PRIMERICA INC,PRI,6311.0,C,00X,LC,10241,22177,1/1/1950,1/30/1962
94,SECURITY PACIFIC NATIONAL BANK,Security Pacific Corporation,6020,12/11/1986,1/17/1991,4205,9577,SECURITY PACIFIC CORP,7448B,6020.0,P,01,LU,60839,4205,12/14/1972,4/30/1992
95,SOUTHERN CALIF S&L ASSOC,California Federal Savings and Loan Association,NaN,6/7/1983,8/5/1983,5884,2612,CALIFORNIA FED BANCORP INC,CAL.2,6035.0,P,01,LC,66456,5884,3/29/1983,1/3/1997
96,TD SECURITIES (USA) LLC,The Toronto-Dominion Bank,"TD (NYSE, TSX)",2/11/2014,Current,29152,15706,TORONTO DOMINION BANK,TD,6020.0,P,01,LC,83835,29152,8/30/1996,E


### Get all data from broker dealers with SIC code 6211 or 6221 that are not in list of primary dealers


In [45]:
linked_bd = linktable[((linktable['sic'] == 6211) | (linktable['sic'] == 6221)) & (~linktable['gvkey'].isin(ticks['gvkey'].tolist()))]
linked_bd

,gvkey,conm,tic,sic,LINKPRIM,LIID,LINKTYPE,LPERMNO,LPERMCO,LINKDT,LINKENDDT
167,1148,ADVEST GROUP INC,ADV.2,6211.0,P,01,LU,64514,420,11/21/1980,2/28/2001
987,1967,BACHE GROUP INC,BAC.,6211.0,P,01,LU,54893,25401,5/31/1972,6/11/1981
1379,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,00X,LU,18877,22881,1/1/1975,1/30/1975
1380,2330,BOSTON CO INC -CL B,BSTNB,6211.0,C,01,LU,18877,22881,1/31/1975,9/30/1981
1475,2432,BROWN (ROBERT C.) & CO INC,RCBI,6211.0,P,01,LU,19538,6441,9/30/1983,11/30/1989
...,...,...,...,...,...,...,...,...,...,...,...
30429,178073,B. RILEY FINANCIAL INC,RILY,6211.0,P,03,LC,15540,55369,7/16/2015,E
30479,178519,LPL FINANCIAL HOLDINGS INC,LPLA,6211.0,P,01,LC,12400,53576,11/18/2010,E
30548,178848,DIRECT MARKETS HOLDINGS CORP,MKTSQ,6211.0,P,01,LC,92312,52781,10/16/2007,9/13/2012
31100,185518,GLOBAL BROKERAGE INC,GLBR,6211.0,C,01,LC,12446,53607,12/2/2010,E


In [46]:
empty_tickers = []
broker_dealers = pd.DataFrame()
for index, row in linked_bd.iterrows():
    gvkey = row['gvkey']
    start_date = '1960-01-01'
    end_date = '2012-12-31'    
    new_data = fetch_financial_data(gvkey, start_date, end_date)
    if isinstance(new_data, tuple):
        empty_tickers.append({row['conm']:gvkey})
    else:
        broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
print(empty_tickers)

C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  broker_dealers = pd.concat([new_data, broker_dealers], axis=0)
C:\Users\jsime\AppData\Local\Temp\ipykernel_19320\1247283076.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA en

[]


In [47]:
broker_dealers.drop_duplicates()

,datadate,total_assets,book_debt,book_equity,market_equity,gvkey,conm
0,1999-12-31,842.252,729.293,NaN,NaN,209386,VAN DER MOOLEN NV
1,2000-12-31,876.311,726.460,NaN,NaN,209386,VAN DER MOOLEN NV
2,2001-12-31,3631.252,3064.614,538.155,1098.808600,209386,VAN DER MOOLEN NV
3,2002-12-31,1567.193,1174.844,327.342,825.624310,209386,VAN DER MOOLEN NV
4,2003-12-31,1114.079,818.679,253.830,333.861110,209386,VAN DER MOOLEN NV
...,...,...,...,...,...,...,...
15,1996-09-30,965.177,875.935,89.242,81.929250,001148,ADVEST GROUP INC
16,1997-09-30,1078.839,973.186,105.653,227.151465,001148,ADVEST GROUP INC
17,1998-09-30,1233.271,1109.604,123.667,182.335875,001148,ADVEST GROUP INC
18,1999-09-30,1462.621,1327.485,135.136,162.881250,001148,ADVEST GROUP INC


Get data from banks
6011 - Federal Reserve Banks
6021 - National Commercial Banks
6022 - State Commercial Banks
6029 - Commercial Banks, nec (not elsewhere classified)
6081 - Foreign Bank and Branches and Agencies
6082 - Foreign Trade and International Banking Institutions

In [51]:
bank_sic = [6011,6021,6022,6029,6081,6082] # these are not all the codes, in the process of looking these up
linked_banks = linktable[(linktable['sic'].isin(bank_sic))]
linked_banks

Series([], Name: conm, dtype: object)